In [3]:
import numpy as np
from scipy import signal, io
import matplotlib.pyplot as plt
from acquisition_tools import cal_matrix, estimate_rpm
from tkinter import Tk
from tkinter.filedialog import askopenfilename
%matplotlib qt

plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 16

cal_mat = cal_matrix()

In [4]:
# Load the .mat file
# Open a file dialog to choose a .mat file
root = Tk()
# root.withdraw()  # Hide the main window
file_path = askopenfilename(initialdir='./results', filetypes=[("MAT files", "*.mat")])
root.destroy()  # Destroy the main window

exp_data = io.loadmat(file_path)

In [32]:
force = exp_data['force']
voltage = exp_data['data']

plt.plot(voltage[:, 0], label='Channel 0')
plt.plot(voltage[:, 1], label='Channel 1')
plt.plot(voltage[:, 2], label='Channel 2')
plt.plot(voltage[:, 3], label='Channel 3')
plt.legend()

In [5]:
time = exp_data['t']
fs_analog = exp_data['Fs_analog']

voltage = exp_data['data']
force_net = exp_data['force_net']
force_raw = exp_data['force']
t_analog = np.linspace(0, len(force_raw)/fs_analog, len(force_raw)).flatten()

z_pulse = voltage[:, 6]
z_pulse = np.where(z_pulse > 1, 5, z_pulse)

rpm_sweep = exp_data['rpm_sweep']
U_inf = exp_data['U_inf']

KeyError: 'Fs_analog'

In [131]:
indices = np.where(np.diff(z_pulse) > 1)[0]
rpm = 60/np.diff(t_analog.flatten()[indices])
t_rpm = t_analog.flatten()[indices[:-1]]

In [132]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, voltage[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, voltage[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, voltage[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, voltage[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, voltage[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, voltage[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [133]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, force_raw[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, force_raw[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, force_raw[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, force_raw[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, force_raw[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, force_raw[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [134]:
dt = 1/fs_analog
order = 2
cutoff = 5
nyq = 0.5 * fs_analog
normal_cutoff = cutoff / nyq

b, a = signal.butter(order, normal_cutoff)
z_l = z_d = signal.lfilter_zi(b, a)

Fx = signal.lfilter(b, a, force_raw[:, 0], zi=z_l)[0]
Fy = signal.lfilter(b, a, force_raw[:, 1], zi=z_l)[0]
Fz = signal.lfilter(b, a, force_raw[:, 2], zi=z_l)[0]
Tx = signal.lfilter(b, a, force_raw[:, 3], zi=z_l)[0]
Ty = signal.lfilter(b, a, force_raw[:, 4], zi=z_l)[0]
Tz = signal.lfilter(b, a, force_raw[:, 5], zi=z_l)[0]

In [139]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, Fx, "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, Fy, "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, Fz, "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, Tx, "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, Tx, "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, Tx, "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [140]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# First subplot
ax1.plot(t_analog, Fz, "-", label='Thrust', linewidth = 2, color='k')
ax1.set_ylim(-20, 20)

ax1_twiny = ax1.twinx()
ax1_twiny.plot(t_rpm, rpm, "-", label='RPM', linewidth=2, color='blue')
ax1_twiny.set_ylabel("RPM")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twiny.get_legend_handles_labels()
ax1_twiny.legend(lines + lines2, labels + labels2, loc='upper left')

ax1.set_xlabel("Time [s]")
ax1.set_ylabel("T [N]")
ax1.grid()

# Second subplot
ax2.plot(t_analog, Tx, "-", label='Torque', linewidth = 2, color='red')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Q [N.m]")
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()